In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
pd.set_option('display.max_columns',None)
from sklearn.feature_selection import SelectFromModel,SelectKBest
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
import random
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

In [4]:
house= pd.read_csv('all_data_clean.csv', na_filter = False)

In [5]:
house.shape

(2553, 82)

In [5]:
ord_colnames = ['LotShape', 'LandSlope', 'OverallQual', 'OverallCond',
                'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
                'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC',
                'KitchenQual', 'Functional', 'FireplaceQu', 'GarageFinish',
                'GarageQual', 'GarageCond', 'PoolQC', 'Fence']
ord_dict = {}
ord_dict['LotShape'] = ['IR3', 'IR2', 'IR1', 'Reg']
ord_dict['LandSlope'] = ['Sev', 'Mod', 'Gtl']
ord_dict['OverallQual'] = [np.nan, 1,2,3,4,5,6,7,8,9,10]
ord_dict['OverallCond'] = [np.nan, 1,2,3,4,5,6,7,8,9,10]
ord_dict['ExterQual'] = [np.nan, 'Po', 'Fa','TA', 'Gd', 'Ex']
ord_dict['ExterCond'] = [np.nan, 'Po', 'Fa','TA', 'Gd', 'Ex']
ord_dict['BsmtQual'] = ['NoneListed', 'Po', 'Fa','TA', 'Gd', 'Ex']
ord_dict['BsmtCond'] = ['NoneListed', 'Po', 'Fa','TA', 'Gd', 'Ex']
ord_dict['BsmtExposure'] = ['NoneListed', 'No', 'Mn', 'Av', 'Gd']
ord_dict['BsmtFinType1'] = ['NoneListed', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ']
ord_dict['BsmtFinType2'] = ['NoneListed', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ']
ord_dict['HeatingQC'] = [np.nan, 'Po', 'Fa','TA', 'Gd', 'Ex']
ord_dict['KitchenQual'] = [np.nan, 'Po', 'Fa','TA', 'Gd', 'Ex']
ord_dict['Functional'] = ['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ']
ord_dict['FireplaceQu'] = ['NoneListed', 'Po', 'Fa','TA', 'Gd', 'Ex']
ord_dict['GarageFinish'] = ['NoneListed', 'Unf', 'RFn', 'Fin']
ord_dict['GarageQual'] = ['NoneListed', 'Po', 'Fa','TA', 'Gd', 'Ex']
ord_dict['GarageCond'] = ['NoneListed', 'Po', 'Fa','TA', 'Gd', 'Ex']
ord_dict['PoolQC'] = ['NoneListed', 'Po', 'Fa','TA', 'Gd', 'Ex']
ord_dict['Fence'] = ['NoneListed', 'MnWw', 'GdWo', 'MnPrv', 'GdPrv']

# Trasforms Labels to Numbers
for i in range(len(ord_colnames)):
    house[ord_colnames[i]] = house[ord_colnames[i]].apply(lambda x: ord_dict[ord_colnames[i]].index(x))

In [8]:
house.head(10)

Unnamed: 0        PID  GrLivArea  SalePrice  MSSubClass MSZoning  \
0           1  909176150        856     126000          30       RL   
1           1  907250050       2000     305900          20       RL   
2           1  909276070       1717     194000          50       RL   
3           2  905476230       1049     139500         120       RL   
4           2  909425140       1839     257500          20       RL   
5           2  534278190        894     117600          20       RL   
6           3  911128020       1001     124900          30  C (all)   
7           3  535301080        882     116000          20       RL   
8           3  534204120       1073     156000          20       RL   
9           4  535377150       1039     114000          70       RL   

   LotFrontage  LotArea Street       Alley  LotShape LandContour LotConfig  \
0         70.0     7890   Pave  NoneListed         3         Lvl    Corner   
1         80.0    10386   Pave  NoneListed         3         Lvl    Inside   
2         80.0    12400   Pave  NoneListed         3         HLS    Inside   
3        110.0     4235   Pave  NoneListed         3         Lvl    Inside   
4         70.0    14442   Pave  NoneListed         3         Lvl    Inside   
5         42.0     8680   Pave  NoneListed         3         Lvl    Inside   
6         60.0     6060   Pave  NoneListed         3         Lvl    Inside   
7         60.0    11556   Pave  NoneListed         3         Lvl    Inside   
8         79.0    10289   Pave  NoneListed         3         Lvl    Inside   
9         80.0     8146   Pave  NoneListed         3         Lvl    Corner   

   LandSlope Neighborhood Condition1 Condition2 BldgType HouseStyle  \
0          2        SWISU       Norm       Norm     1Fam     1Story   
1          2      CollgCr       Norm       Norm     1Fam     1Story   
2          1      Crawfor       Norm       Norm     1Fam     1.5Fin   
3          2      Edwards       Norm       Norm   TwnhsE     1Story   
4          2      Crawfor       Norm       Norm     1Fam     1Story   
5          2        NAmes       Norm       Norm     1Fam     1Story   
6          2       IDOTRR       Norm       Norm     1Fam     1Story   
7          2        NAmes       Norm       Norm     1Fam     1Story   
8          2        NAmes       Norm       Norm     1Fam     1Story   
9          2      OldTown       Norm       Norm     1Fam     2Story   

   OverallQual  OverallCond  YearBuilt  YearRemodAdd RoofStyle RoofMatl  \
0            6            6       1939          1950     Gable  CompShg   
1            8            5       2004          2005     Gable  CompShg   
2            5            6       1940          1950     Gable  CompShg   
3            5            5       1984          1984     Gable  CompShg   
4            6            7       1957          2004       Hip  CompShg   
5            5            7       1960          1960       Hip  CompShg   
6            5            9       1930          2007       Hip  CompShg   
7            5            6       1952          1952     Gable  CompShg   
8            5            7       1965          1965       Hip  CompShg   
9            4            8       1900          2003     Gable  CompShg   

  Exterior1st Exterior2nd MasVnrType  MasVnrArea  ExterQual  ExterCond  \
0     Wd Sdng     Wd Sdng       None         0.0          3          3   
1     CemntBd     CmentBd      Stone       246.0          4          3   
2     Wd Sdng     Wd Sdng       None         0.0          3          3   
3     HdBoard     HdBoard    BrkFace       149.0          4          3   
4     CemntBd     CmentBd    BrkFace       106.0          3          3   
5     CemntBd     CmentBd       None         0.0          3          3   
6     MetalSd     MetalSd       None         0.0          4          3   
7     MetalSd     MetalSd      Stone        52.0          3          3   
8     MetalSd     MetalSd    BrkFace       168.0          3          3   
9     MetalSd 

In [6]:
house['MSSubClass'] = house['MSSubClass'].astype(str)

In [7]:
house.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2553 entries, 0 to 2552
Data columns (total 82 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       2553 non-null   int64  
 1   PID              2553 non-null   int64  
 2   GrLivArea        2553 non-null   int64  
 3   SalePrice        2553 non-null   int64  
 4   MSSubClass       2553 non-null   object 
 5   MSZoning         2553 non-null   object 
 6   LotFrontage      2553 non-null   float64
 7   LotArea          2553 non-null   int64  
 8   Street           2553 non-null   object 
 9   Alley            2553 non-null   object 
 10  LotShape         2553 non-null   int64  
 11  LandContour      2553 non-null   object 
 12  LotConfig        2553 non-null   object 
 13  LandSlope        2553 non-null   int64  
 14  Neighborhood     2553 non-null   object 
 15  Condition1       2553 non-null   object 
 16  Condition2       2553 non-null   object 
 17  BldgType      

#### Setting up variables to use for modeling

In [ ]:
#Numericals

#SalePrice
num = house[['GrLivArea', 'LotFrontage', 'LotArea',
       'LotShape', 'LandSlope', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'MasVnrArea', 'ExterQual', 'ExterCond', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'HeatingQC',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'BsmtFullBath', 'BsmtHalfBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscVal', 'MoSold',
       'YrSold']]

#SalePricePerGLA
num1 = house[['LotFrontage', 'LotArea',
       'LotShape', 'LandSlope', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'MasVnrArea', 'ExterQual', 'ExterCond', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'HeatingQC',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'BsmtFullBath', 'BsmtHalfBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscVal', 'MoSold',
       'YrSold']]

In [8]:
Categoricals

cat= house.dtypes[house.dtypes == "object"].index

categories = house[cat]

cat_dummies = pd.get_dummies(house[cat],drop_first=True)

cat_dummies= cat_dummies.astype(object)

In [10]:
X = pd.concat((cat_dummies, num),axis=1)
X1 = pd.concat((cat_dummies, num1),axis=1)

In [11]:
#target variables
y= house['SalePricePerGLA']
y1= house['SalePrice']
y2 = np.log(house['SalePrice'])
y3 = np.log(house['SalePricePerGLA'])

In [12]:
from sklearn.model_selection import train_test_split, cross_val_score
import random

np.random.seed(10)

X_train, X_test, y_train, y_test = train_test_split(X, y2, test_size=0.25, random_state=21) #SalePrice

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y3, test_size=0.25, random_state=21) #log SalePricePerGLA

X2_train, X2_test, y2_train, y2_test = train_test_split(X1, y, test_size=0.25, random_state=21) #SalePricePerGLA


#### Scaled variables/ SalePrice: SVR- rbf

In [13]:
#Scale values
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_trains = scaler.fit_transform(X_train)
X_tests = scaler.transform(X_test)
X_train = pd.DataFrame(X_trains, index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(X_tests, index=X_test.index, columns=X_test.columns)

In [291]:
#Grid
from sklearn.svm import SVR
import sklearn.model_selection as ms
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

svr = SVR()
svr.set_params(kernel='rbf')

grid = {'C':[.1, 1, 10, 100, 1000], 
              'gamma' : [0.00001, 0.0001, 0.001, .01], 
              'epsilon':[0.1,0.2,0.3,0.4,0.5]}

cv = KFold(n_splits=10, shuffle= True, random_state=42)

grid = ms.GridSearchCV(svr, grid, cv=cv, return_train_score = True, n_jobs = -1)

In [292]:
#results
result = grid.fit(X_train, y_train)

print(grid.best_score_)
print(grid.best_estimator_.score(X_test, y_test))
print(result.best_params_)

0.9183627624392644
0.9205428197441602
{'C': 10, 'epsilon': 0.1, 'gamma': 0.0001}


#### Scaled variables/ Log SalePricePerGLA: SVR- rbf

In [14]:
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X1_train)
X_tests1 = scaler.transform(X1_test)
X1_train = pd.DataFrame(X_train1, index=X1_train.index, columns=X1_train.columns)
X1_test = pd.DataFrame(X_tests1, index=X1_test.index, columns=X1_test.columns)

In [295]:
#Same grid but for logSalePrice

result = grid.fit(X1_train, y1_train)

print(grid.best_score_) #0.82510
print(grid.best_estimator_.score(X1_test, y1_test)) #0.79242
print(result.best_params_) #{'C': 1000, 'epsilon': 0.1, 'gamma': 1e-05}

KeyboardInterrupt: 

scaled/ spgla - rbf

In [15]:
scaler = StandardScaler()
X_train2 = scaler.fit_transform(X2_train)
X_tests2 = scaler.transform(X2_test)
X2_train = pd.DataFrame(X_train2, index=X2_train.index, columns=X2_train.columns)
X2_test = pd.DataFrame(X_tests2, index=X2_test.index, columns=X2_test.columns)

In [296]:
#Same Grid but for SalePricePerGLA

result = grid.fit(X2_train, y2_train)

print(grid.best_score_) #0.81833
print((grid.best_estimator_.score(X2_test, y2_test)) #0.81774
print(result.best_params_) #{'C': 1000, 'epsilon': 0.5, 'gamma': 0.0001}

KeyboardInterrupt: 

scaled/ spgla - sigmoid

In [ ]:
#Different Gridsearch for Sigmoid
from sklearn.svm import SVR
import sklearn.model_selection as ms
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

svr = SVR()
svr.set_params(kernel='sigmoid')

grid = {'C':[.1, 1, 10, 100, 1000], 
              'gamma' : [0.00001, 0.0001, 0.001, .01], 
              'epsilon':[0.1,0.2,0.3,0.4,0.5]}

cv = KFold(n_splits=10, shuffle= True, random_state=42)

grid = ms.GridSearchCV(svr, grid, cv=cv, return_train_score = True, n_jobs = -1)

In [297]:
result = grid.fit(X2_train, y2_train)

print(grid.best_score_) #0.79310
print(grid.best_estimator_.score(X2_test, y2_test)) #0.79868
print(result.best_params_) # {'C': 100, 'epsilon': 0.1, 'gamma': 0.001}

KeyboardInterrupt: 

#### Scaled/ spgla - linear

In [16]:
#Different Grid for SVR
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

svr = SVR()
svr_params = {'kernel': ['linear'], 'gamma': [1e-4, 1e-3, 0.01, 0.1],'C': [.01,.1, 1, 10],'epsilon': [1e-3, 0.01, 0.1, 0.2, 0.5]}
svr_cv = GridSearchCV(svr,svr_params, cv=10)

In [ ]:
svr_cv.fit(X2_train, y2_train)

print(svr_cv.best_score_) #0.8097381987878937
print(svr_cv.best_estimator_.score(X2_test, y2_test)) #0.7983250776664886

In [28]:
svr_cv.best_params_ #{'C': 1, 'epsilon': 0.5, 'gamma': 0.0001, 'kernel': 'linear'} 

{'C': 1, 'epsilon': 0.5, 'gamma': 0.0001, 'kernel': 'linear'}

#### Scaled/ log spgla - linear

In [60]:
svr_cv.fit(X1_train, y1_train)
print(svr_cv.best_score_(X1_train, y1_train))
print(svr_cv.best_estimator_.score(X1_test, y1_test)) #log

0.8685210496104686
0.7889213932543027


Decided to go with SVR - Linear: {'C': 1, 'epsilon': 0.5, 'gamma': 0.0001, 'kernel': 'linear'}

#### Top features: Linear, scaled/spgla

In [18]:
svm= SVR(C=1, epsilon=0.5, gamma= 0.0001, kernel= 'linear')
svm.fit(X2_train, y2_train)

SVR(C=1, epsilon=0.5, gamma=0.0001, kernel='linear')

In [38]:
a= svm.coef_.reshape(-1,1)
b= pd.DataFrame(a, index= X2_train.columns, columns= ['Coefs']).sort_values(by= 'Coefs', ascending=False).reset_index()
b= b.rename(columns={"index": "Features"})
b
#top= pd.DataFrame(a, index= X2_train.columns, columns= ['Coefs']).sort_values(by= 'Coefs', ascending=False).head(10)

Features      Coefs
0            OverallQual   6.123672
1              YearBuilt   5.962527
2            OverallCond   5.253630
3      HouseStyle_1Story   4.808384
4        Condition1_Norm   3.564337
..                   ...        ...
210  Exterior2nd_HdBoard  -2.243610
211        MSSubClass_60  -2.787500
212       MSSubClass_160  -3.446482
213             2ndFlrSF  -9.628135
214             1stFlrSF -13.742777

[215 rows x 2 columns]

In [61]:
bottom= pd.DataFrame(a, index= X2_train.columns, columns= ['Coefs']).sort_values(by= 'Coefs', ascending=False).tail(10)

In [71]:
#df3 = pd.concat((top,bottom),axis=0).reset_index()
#df3.plot(x='index', y='Coefs',kind='barh',figsize=(11,11), title= 'SVR - Linear: Coefficients',
        #xlabel='Coefficients', ylabel= 'Features')

In [75]:
df3.rename(columns={"index": "Features", 'Coefs': 'Coeficients'})

Features  Coeficients
0            OverallQual     6.123672
1              YearBuilt     5.962527
2            OverallCond     5.253630
3      HouseStyle_1Story     4.808384
4        Condition1_Norm     3.564337
5             BsmtFinSF1     3.511294
6                LotArea     3.434646
7      HouseStyle_SFoyer     3.340436
8   Neighborhood_NridgHt     3.181820
9          MSSubClass_20     3.000201
10   Exterior1st_CemntBd    -1.307272
11   Exterior2nd_Plywood    -1.606852
12      MSZoning_C (all)    -1.790347
13           MSZoning_RM    -1.834059
14  Neighborhood_Edwards    -1.867012
15   Exterior2nd_HdBoard    -2.243610
16         MSSubClass_60    -2.787500
17        MSSubClass_160    -3.446482
18              2ndFlrSF    -9.628135
19              1stFlrSF   -13.742777

#### Top Changeable Features

In [72]:
unchangeable_feats = ([feat for feat in list(b.Features) if 'Neighborhood' in feat] +
 [feat for feat in list(b.Features) if 'Condition1' in feat] +
 [feat for feat in list(b.Features) if 'Condition2' in feat] +
 [feat for feat in list(b.Features) if 'Exterior1st' in feat] +
 [feat for feat in list(b.Features) if 'MSSubClass' in feat] +
 [feat for feat in list(b.Features) if 'SaleCondition' in feat] +
 [feat for feat in list(b.Features) if 'GarageType' in feat] +
 [feat for feat in list(b.Features) if 'LandContour' in feat] +
 [feat for feat in list(b.Features) if 'MSZoning' in feat] +
 [feat for feat in list(b.Features) if 'LotShape' in feat] + 
 [feat for feat in list(b.Features) if 'HouseStyle' in feat] +
 [feat for feat in list(b.Features) if 'ExterQual' in feat] +                      
 [feat for feat in list(b.Features) if 'const' in feat] +                      
 [feat for feat in list(b.Features) if 'YearBuilt' in feat] +                      
 [feat for feat in list(b.Features) if '2ndFlrSF' in feat] +                      
 [feat for feat in list(b.Features) if '1stFlrSF' in feat] +                     
 [feat for feat in list(b.Features) if 'GarageYrBlt' in feat] +
 [feat for feat in list(b.Features) if 'GarageArea' in feat] +                    
 [feat for feat in list(b.Features) if 'LotArea' in feat] +                      
 [feat for feat in list(b.Features) if 'TotalBsmtSF' in feat] + 
 [feat for feat in list(b.Features) if 'MoSold' in feat] + 
 [feat for feat in list(b.Features) if 'YrSold' in feat]+
 [feat for feat in list(b.Features) if 'PoolQC' in feat] +
 [feat for feat in list(b.Features) if 'BldgType' in feat]+
 [feat for feat in list(b.Features) if 'LotConfig' in feat]+                      
 [feat for feat in list(b.Features) if 'GarageCars' in feat]+
 [feat for feat in list(b.Features) if 'YearRemodAdd' in feat])

In [119]:
changeable_cols = [feat for feat in list(b['Features']) if feat not in unchangeable_feats]
feat_importance = b[b['Features'].isin(changeable_cols)]
v= feat_importance.tail(4)
f= feat_importance.head(17)
f= f.merge(v, how='outer')

f.plot(x='Features', y='Coefs',kind='barh',figsize=(11,11), title= 'SVR- Linear: Improvable Coefficients',
        xlabel='Coefficients', ylabel= 'Features')

<AxesSubplot:title={'center':'SVR(Linear): Improvable Coefficients'}, ylabel='Coefficients'>

#### Get total Predictions from Model

In [82]:
svm= SVR(C=1, epsilon =0.5, gamma= 0.0001, kernel = 'linear')

In [100]:
all_x = pd.concat((X2_train, X2_test),axis=0)
all_y = pd.concat((y2_train, y2_test),axis=0)
ttl_preds = list(zip(all_y.index,svm.predict(all_x)))

#### We decided to look at 80 % of houses with lowest GrLivArea or GLA

In [101]:
top_twenty = list(house.nlargest((int(len(X)*.2)),'GrLivArea').index)
bottom_eighty = list(house.nsmallest((int(len(X)*.8)),'GrLivArea').index)
house8= house.iloc[bottom_eighty]

In [102]:
pred_df = pd.DataFrame((np.array(ttl_preds)),columns=['ignore','Prediction']).drop('ignore',axis=1)

In [103]:
house8= house8.merge(pred_df,left_index=True,right_index=True)

In [104]:
house8

Unnamed: 0        PID  GrLivArea  SalePrice MSSubClass MSZoning  \
2472         918  534450090        334      39300         20       RL   
2409         855  902205070        407      46500         20       RM   
1636         551  909175080        438      60000         30       RL   
1913         645  911175430        480      35311         20  C (all)   
111           38  902206020        492      61500         30       RM   
...          ...        ...        ...        ...        ...      ...   
990          334  903429080       1826     155000         70       RM   
405          137  903206160       1827     202500         70       RL   
1289         434  910201180       1828     150909         70       RM   
2081         701  907275140       1828     279500         20       RL   
395          133  534129040       1829     218500         20       RL   

      LotFrontage  LotArea Street       Alley  LotShape LandContour LotConfig  \
2472         50.0     5000   Pave  NoneListed         3         Low    Inside   
2409         62.0     7311   Pave  NoneListed         3         Lvl    Corner   
1636         60.0     8400   Pave  NoneListed         3         Bnk    Inside   
1913         50.0     9000   Pave  NoneListed         3         Lvl    Inside   
111          52.0     7830   Pave  NoneListed         3         Lvl    Inside   
...           ...      ...    ...         ...       ...         ...       ...   
990          70.0     2500   Pave        Pave         3         Lvl    Corner   
405          69.0    11737   Pave  NoneListed         2         Bnk    Inside   
1289         50.0     9060   Pave  NoneListed         3         Lvl    Inside   
2081         70.0    12782   Pave  NoneListed         3         Lvl    Inside   
395          70.0    10456   Pave  NoneListed         2         Lvl    Inside   

      LandSlope Neighborhood Condition1 Condition2 BldgType HouseStyle  \
2472          1      BrkSide       Norm       Norm     1Fam     1Story   
2409          2      OldTown       Norm       Norm     1Fam     1Story   
1636          2        SWISU       Norm       Norm     1Fam     1Story   
1913          2       IDOTRR       Norm       Norm     1Fam     1Story   
111           2      OldTown       Norm       Norm     1Fam     1Story   
...         ...          ...        ...        ...      ...        ...   
990           2      OldTown       Norm       Norm     1Fam     2Story   
405           2      BrkSide       Norm       Norm     1Fam     2Story   
1289          2       IDOTRR       Norm       Norm     1Fam     2Story   
2081          2      CollgCr       Norm       Norm     1Fam     1Story   
395           2       NWAmes       Norm       Norm     1Fam     1Story   

      OverallQual  OverallCond  YearBuilt  YearRemodAdd RoofStyle RoofMatl  \
2472            1            3       1946          1950     Gable  CompShg   
2409            2            5       1946          1950     Gable  CompShg   
1636            2            5       1920          1950     Gable  CompShg   
1913            2            3       1949          1950     Gable  CompShg   
111             3            5       1921          1950     Gable  CompShg   
...           ...          ...        ...           ...       ...      ...   
990             7            8       1915          2005     Gable  CompShg   
405             6            7       1924          1996   Gambrel  CompShg   
1289            5            6       1923          1999     Gable  CompShg   
2081            8            5       2002          2003       Hip  CompShg   
395             6            6       1967          1967       Hip  CompShg   

     Exterior1st Exterior2nd MasVnrType  MasVnrArea  ExterQual  ExterCond  \
2472     VinylSd     VinylSd       None         0.0          2          2   
2409     VinylSd     VinylSd       None         0.0          3          4   
1636     Wd Sdng     Wd Sdng       None         0.0          3          3   
1913     AsbShng     A

In [105]:
house8['Predicted SalePrice'] = house8['Prediction'] * house8['GrLivArea']

In [106]:
avg_nh = pd.DataFrame(house8.groupby('Neighborhood')[['SalePricePerGLA']].agg(['mean',np.std]))

In [107]:
avg_nh.columns=['Avg_by_NH', 'SD_by_NH']
avg_nh = avg_nh.reset_index()

underval_start = pd.merge(house8,avg_nh,left_on='Neighborhood',right_on='Neighborhood')

In [109]:
underval_start['Avg_by_NH']= underval_start['Avg_by_NH'] * underval_start['GrLivArea']
underval_start['SD_by_NH']= underval_start['SD_by_NH'] * underval_start['GrLivArea']

In [110]:
underval_start['SDs_away'] = underval_start['SalePrice']/(underval_start['Avg_by_NH'] - underval_start['SD_by_NH'])
undervalued = underval_start[underval_start['SDs_away']<1]
overval = underval_start[underval_start['SDs_away']>1]

In [115]:
undervalued

Unnamed: 0        PID  GrLivArea  SalePrice MSSubClass MSZoning  \
6            550  534451170        729      52000         30       RL   
10           457  903231060        800      60000         30       RM   
24           182  903204095        935      79500         40       RM   
38           407  903228070       1142      93000         50       RM   
54           375  903225160       1290     106900         70       RM   
...          ...        ...        ...        ...        ...      ...   
2011          37  528188090       1626     155000        160       RL   
2012         256  528188060       1626     173000        160       RL   
2013         262  528188120       1626     175500        160       RL   
2014         807  528188150       1626     154000        160       RL   
2041          60  533135020       1755     235000         60       RL   

      LotFrontage  LotArea Street       Alley  LotShape LandContour LotConfig  \
6            56.0     4130   Pave  NoneListed         2         Lvl    Inside   
10           51.0     6120   Pave  NoneListed         3         Lvl    Inside   
24           50.0     9100   Pave  NoneListed         3         Lvl    Inside   
38           60.0     6000   Pave  NoneListed         3         Lvl    Corner   
54           59.0     5870   Pave  NoneListed         3         Lvl    Corner   
...           ...      ...    ...         ...       ...         ...       ...   
2011         50.0     2448   Pave  NoneListed         3         Lvl    Inside   
2012         98.0     2268   Pave  NoneListed         3         Lvl    Inside   
2013         85.0     2628   Pave  NoneListed         3         Lvl    Inside   
2014         36.0     2448   Pave  NoneListed         3         Lvl    Inside   
2041         70.0    11949   Pave  NoneListed         3         Lvl    Inside   

      LandSlope Neighborhood Condition1 Condition2 BldgType HouseStyle  \
6             2      BrkSide       Norm       Norm     1Fam     1Story   
10            2      BrkSide       Norm       Norm     1Fam     1Story   
24            2      BrkSide       RRAn      Feedr     1Fam     1Story   
38            2      BrkSide       Norm       Norm     1Fam     1.5Fin   
54            2      BrkSide      Feedr      Feedr     1Fam     2Story   
...         ...          ...        ...        ...      ...        ...   
2011          2      NridgHt       Norm       Norm    Twnhs     2Story   
2012          2      NridgHt       Norm       Norm    Twnhs     2Story   
2013          2      NridgHt       Norm       Norm    Twnhs     2Story   
2014          2      NridgHt       Norm       Norm    Twnhs     2Story   
2041          2      NoRidge       Norm       Norm     1Fam     2Story   

      OverallQual  OverallCond  YearBuilt  YearRemodAdd RoofStyle RoofMatl  \
6               3            6       1935          2003     Gable  CompShg   
10              2            3       1936          1950     Gable  CompShg   
24              5            6       1930          1960     Gable  CompShg   
38              5            5       1924          1950     Gable  CompShg   
54              6            9       1900          2000     Gable  CompShg   
...           ...          ...        ...           ...       ...      ...   
2011            7            5       2003          2004     Gable  CompShg   
2012            7            5       2003          2004     Gable  CompShg   
2013            7            5       2003          2003     Gable  CompShg   
2014            7            5       2003          2004     Gable  CompShg   
2041            7            8       1991          2008     Gable  CompShg   

     Exterior1st Exterior2nd MasVnrType  MasVnrArea  ExterQual  ExterCond  \
6        VinylSd     VinylSd       None         0.0          4          3   
10       AsbShng     AsbShng       None         0.0          2          2   
24       VinylSd     VinylSd       None         0.0          3          3   
38        Stucco      

#### Train new model with the 80%

In [128]:
num_u = undervalued[['LotFrontage', 'LotArea',
       'LotShape', 'LandSlope', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'MasVnrArea', 'ExterQual', 'ExterCond', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'HeatingQC',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'BsmtFullBath', 'BsmtHalfBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscVal', 'MoSold',
       'YrSold']]

In [129]:
cat= undervalued.dtypes[undervalued.dtypes == "object"].index

categories = undervalued[cat]

cat_dummies = pd.get_dummies(undervalued[cat],drop_first=True)


In [131]:
X = pd.concat((cat_dummies, num_u),axis=1)
y= undervalued['SalePricePerGLA']

In [154]:
from sklearn.model_selection import train_test_split, cross_val_score
import random

np.random.seed(10)

X_train_u, X_test_u, y_train_u, y_test_u = train_test_split(X, y, test_size=0.25, random_state=21)

In [155]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_trains_u = scaler.fit_transform(X_train_u)
X_tests_u = scaler.transform(X_test_u)
X_train_u = pd.DataFrame(X_trains_u, index=X_train_u.index, columns=X_train_u.columns)
X_test_u = pd.DataFrame(X_tests_u, index=X_test_u.index, columns=X_test_u.columns)

In [138]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

svr = SVR()
svr_params = {'kernel': ['linear'], 'gamma': [1e-4, 1e-3, 0.01, 0.1],'C': [.01,.1, 1, 10],'epsilon': [1e-3, 0.01, 0.1, 0.2, 0.5]}
svr_cv = GridSearchCV(svr,svr_params, cv=10)

In [160]:
svr_cv.fit(X_train_u, y_train_u)

svr_cv.best_params_
#{'C': [0.1],'epsilon': [0.5],'gamma': [0.0001],'kernel': ['linear']})

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.01, 0.1, 1, 10],
                         'epsilon': [0.001, 0.01, 0.1, 0.2, 0.5],
                         'gamma': [0.0001, 0.001, 0.01, 0.1],
                         'kernel': ['linear']})

In [162]:
svm = SVR(C=0.1, epsilon= 0.5, gamma= 0.0001, kernel= 'linear')

In [163]:
svm.fit(X_train_u, y_train_u)

SVR(C=0.1, epsilon=0.5, gamma=0.0001, kernel='linear')

In [168]:
print(svm.score(X_train_u, y_train_u))
print(svm.score(X_test_u, y_test_u))

0.8884763686785758
0.733295767882613


#### Setting up an improved dataset to see if new predictions are greater.

In [188]:
improved= undervalued.drop('Unnamed: 0', axis=1)

In [220]:
improved

PID  GrLivArea  SalePrice MSSubClass MSZoning  LotFrontage  \
6     534451170        729      52000         30       RL         56.0   
10    903231060        800      60000         30       RM         51.0   
24    903204095        935      79500         40       RM         50.0   
38    903228070       1142      93000         50       RM         60.0   
54    903225160       1290     106900         70       RM         59.0   
...         ...        ...        ...        ...      ...          ...   
2011  528188090       1626     155000        160       RL         50.0   
2012  528188060       1626     173000        160       RL         98.0   
2013  528188120       1626     175500        160       RL         85.0   
2014  528188150       1626     154000        160       RL         36.0   
2041  533135020       1755     235000         60       RL         70.0   

      LotArea Street       Alley  LotShape LandContour LotConfig  LandSlope  \
6        4130   Pave  NoneListed         2         Lvl    Inside          2   
10       6120   Pave  NoneListed         3         Lvl    Inside          2   
24       9100   Pave  NoneListed         3         Lvl    Inside          2   
38       6000   Pave  NoneListed         3         Lvl    Corner          2   
54       5870   Pave  NoneListed         3         Lvl    Corner          2   
...       ...    ...         ...       ...         ...       ...        ...   
2011     2448   Pave  NoneListed         3         Lvl    Inside          2   
2012     2268   Pave  NoneListed         3         Lvl    Inside          2   
2013     2628   Pave  NoneListed         3         Lvl    Inside          2   
2014     2448   Pave  NoneListed         3         Lvl    Inside          2   
2041    11949   Pave  NoneListed         3         Lvl    Inside          2   

     Neighborhood Condition1 Condition2 BldgType HouseStyle  OverallQual  \
6         BrkSide       Norm       Norm     1Fam     1Story            9   
10        BrkSide       Norm       Norm     1Fam     1Story            9   
24        BrkSide       RRAn      Feedr     1Fam     1Story            9   
38        BrkSide       Norm       Norm     1Fam     1.5Fin            9   
54        BrkSide      Feedr      Feedr     1Fam     2Story            9   
...           ...        ...        ...      ...        ...          ...   
2011      NridgHt       Norm       Norm    Twnhs     2Story            9   
2012      NridgHt       Norm       Norm    Twnhs     2Story            9   
2013      NridgHt       Norm       Norm    Twnhs     2Story            9   
2014      NridgHt       Norm       Norm    Twnhs     2Story            9   
2041      NoRidge       Norm       Norm     1Fam     2Story            9   

      OverallCond  YearBuilt  YearRemodAdd RoofStyle RoofMatl Exterior1st  \
6               9       1935          2003     Gable  CompShg     VinylSd   
10              9       1936          1950     Gable  CompShg     AsbShng   
24              9       1930          1960     Gable  CompShg     VinylSd   
38              9       1924          1950     Gable  CompShg      Stucco   
54              9       1900          2000     Gable  CompShg     HdBoard   
...           ...        ...           ...       ...      ...         ...   
2011            9       2003          2004     Gable  CompShg     VinylSd   
2012            9       2003          2004     Gable  CompShg     VinylSd   
2013            9       2003          2003     Gable  CompShg     VinylSd   
2014            9       2003          2004     Gable  CompShg     VinylSd   
2041            9       1991          2008     Gable  CompShg     VinylSd   

     Exterior2nd MasVnrType  MasVnrArea  ExterQual  ExterCond Foundation  \
6        VinylSd       None         0.0          4          3     CBlock   
10       AsbShng       None         0.0          2          2     BrkTil   
24       VinylSd       None         0.0          3          3     BrkTil   
38        Stucco    BrkFace     

Adjusting Features

In [190]:
improved['OverallQual']=improved.loc[:,'OverallQual'] = 9
improved['OverallCond']=improved.loc[:,'OverallCond'] = 9
improved['BsmtExposure']=improved.loc[:,'BsmtExposure'] = 3
improved['BsmtQual']=improved.loc[:,'BsmtQual'] = 4
improved['KitchenQual']=improved.loc[:,'KitchenQual'] = 4
improved['Functional']=improved.loc[:,'Functional'] = 7
improved['FireplaceQu']=improved.loc[:,'FireplaceQu'] = 4

In [191]:
improved['BsmtFinSF1']= improved.loc[:,'BsmtFinSF1'] = improved['BsmtFinSF1'] + (improved['BsmtUnfSF']/ 2)

In [192]:
improved['BsmtFullBath']= improved.loc[:,'BsmtFullBath'] = improved['BsmtFullBath'] + 1

In [193]:
improved['BsmtUnfSF']= improved.loc[:,'BsmtUnfSF'] = improved['BsmtUnfSF'] - (improved['BsmtUnfSF']/ 2)

In [195]:
cat= improved.dtypes[improved.dtypes == "object"].index

categories = improved[cat]

cat_dummies = pd.get_dummies(improved[cat],drop_first=True)

In [196]:
cat_dummies

MSSubClass_150  MSSubClass_160  MSSubClass_190  MSSubClass_20  \
6                  0               0               0              0   
10                 0               0               0              0   
24                 0               0               0              0   
38                 0               0               0              0   
54                 0               0               0              0   
...              ...             ...             ...            ...   
2011               0               1               0              0   
2012               0               1               0              0   
2013               0               1               0              0   
2014               0               1               0              0   
2041               0               0               0              0   

      MSSubClass_30  MSSubClass_40  MSSubClass_45  MSSubClass_50  \
6                 1              0              0              0   
10                1              0              0              0   
24                0              1              0              0   
38                0              0              0              1   
54                0              0              0              0   
...             ...            ...            ...            ...   
2011              0              0              0              0   
2012              0              0              0              0   
2013              0              0              0              0   
2014              0              0              0              0   
2041              0              0              0              0   

      MSSubClass_60  MSSubClass_70  MSSubClass_80  MSSubClass_85  \
6                 0              0              0              0   
10                0              0              0              0   
24                0              0              0              0   
38                0              0              0              0   
54                0              1              0              0   
...             ...            ...            ...            ...   
2011              0              0              0              0   
2012              0              0              0              0   
2013              0              0              0              0   
2014              0              0              0              0   
2041              1              0              0              0   

      MSSubClass_90  MSZoning_C (all)  MSZoning_FV  MSZoning_RH  MSZoning_RL  \
6                 0                 0            0            0            1   
10                0                 0            0            0            0   
24                0                 0            0            0            0   
38                0                 0            0            0            0   
54                0                 0            0            0            0   
...             ...               ...          ...          ...          ...   
2011              0                 0            0            0            1   
2012              0                 0            0            0            1   
2013              0                 0            0            0            1   
2014              0                 0            0            0            1   
2041              0                 0            0            0            1   

      MSZoning_RM  Alley_NoneListed  Alley_Pave  LandContour_HLS  \
6               0                 1           0                0   
10              1                 1           0                0   
24              1                 1           0                0   
38              1                 1           0                0   
54              1                 1           0                0   
...           ...               ...         ...              ...   
2011            0                 1           0                0   
2012          

In [197]:
cat_dummies['MasVnrType_None']=cat_dummies.loc[:,'MasVnrType_None'] = 1
cat_dummies['Exterior2nd_CmentBd']=cat_dummies.loc[:,'Exterior2nd_CmentBd'] = 1
cat_dummies['MiscFeature_Shed']=cat_dummies.loc[:,'MiscFeature_Shed'] = 1

In [199]:
num_i = improved[['LotFrontage', 'LotArea',
       'LotShape', 'LandSlope', 'OverallQual', 'OverallCond', 'YearBuilt',
       'YearRemodAdd', 'MasVnrArea', 'ExterQual', 'ExterCond', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'HeatingQC',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'BsmtFullBath', 'BsmtHalfBath',
       'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscVal', 'MoSold',
       'YrSold']]

In [200]:
X = pd.concat((cat_dummies, num_i),axis=1)
y= improved['SalePricePerGLA']

In [204]:
from sklearn.model_selection import train_test_split, cross_val_score
import random

np.random.seed(10)

X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(X, y, test_size=0.25, random_state=21)

In [205]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_trains_i = scaler.fit_transform(X_train_i)
X_tests_i = scaler.transform(X_test_i)
X_train_i = pd.DataFrame(X_trains_i, index=X_train_u.index, columns=X_train_i.columns)
X_test_i = pd.DataFrame(X_tests_i, index=X_test_u.index, columns=X_test_i.columns)

In [231]:
X_test_i

MSSubClass_150  MSSubClass_160  MSSubClass_190  MSSubClass_20  \
1116       -0.065938       -0.378899       -0.148741      -0.422577   
1824       -0.065938        2.639227       -0.148741      -0.422577   
1742       -0.065938       -0.378899       -0.148741      -0.422577   
1146       -0.065938        2.639227       -0.148741      -0.422577   
1759       -0.065938       -0.378899       -0.148741      -0.422577   
...              ...             ...             ...            ...   
1249       -0.065938       -0.378899       -0.148741       2.366432   
1604       -0.065938       -0.378899       -0.148741      -0.422577   
1366       -0.065938       -0.378899       -0.148741      -0.422577   
1338       -0.065938       -0.378899       -0.148741      -0.422577   
1039       -0.065938       -0.378899       -0.148741      -0.422577   

      MSSubClass_30  MSSubClass_40  MSSubClass_45  MSSubClass_50  \
1116      -0.272798      -0.065938            1.0      -0.485071   
1824      -0.272798      -0.065938            0.0      -0.485071   
1742      -0.272798      -0.065938            0.0      -0.485071   
1146      -0.272798      -0.065938            0.0      -0.485071   
1759      -0.272798      -0.065938            0.0      -0.485071   
...             ...            ...            ...            ...   
1249      -0.272798      -0.065938            0.0      -0.485071   
1604      -0.272798      -0.065938            0.0      -0.485071   
1366      -0.272798      -0.065938            0.0      -0.485071   
1338      -0.272798      -0.065938            0.0      -0.485071   
1039      -0.272798      -0.065938            0.0       2.061553   

      MSSubClass_60  MSSubClass_70  MSSubClass_80  MSSubClass_85  \
1116      -0.498647      -0.263523      -0.065938      -0.065938   
1824      -0.498647      -0.263523      -0.065938      -0.065938   
1742      -0.498647      -0.263523      -0.065938      -0.065938   
1146      -0.498647      -0.263523      -0.065938      -0.065938   
1759       2.005427      -0.263523      -0.065938      -0.065938   
...             ...            ...            ...            ...   
1249      -0.498647      -0.263523      -0.065938      -0.065938   
1604       2.005427      -0.263523      -0.065938      -0.065938   
1366       2.005427      -0.263523      -0.065938      -0.065938   
1338       2.005427      -0.263523      -0.065938      -0.065938   
1039      -0.498647      -0.263523      -0.065938      -0.065938   

      MSSubClass_90  MSZoning_C (all)  MSZoning_FV  MSZoning_RH  MSZoning_RL  \
1116      -0.386334         -0.163299    -0.234082     8.717798    -1.581139   
1824      -0.386334         -0.163299     4.272002    -0.114708    -1.581139   
1742       2.588436         -0.163299    -0.234082    -0.114708     0.632456   
1146      -0.386334         -0.163299    -0.234082    -0.114708     0.632456   
1759      -0.386334         -0.163299    -0.234082    -0.114708     0.632456   
...             ...               ...          ...          ...          ...   
1249      -0.386334         -0.163299    -0.234082    -0.114708     0.632456   
1604      -0.386334         -0.163299    -0.234082    -0.114708     0.632456   
1366      -0.386334         -0.163299    -0.234082    -0.114708     0.632456   
1338      -0.386334         -0.163299    -0.234082    -0.114708     0.632456   
1039      -0.386334         -0.163299    -0.234082    -0.114708     0.632456   

      MSZoning_RM  Alley_NoneListed  Alley_Pave  LandContour_HLS  \
1116    -0.485071          0.299370   -0.163299        -0.114708   
1824    -0.485071         -3.340344    6.123724        -0.114708   
1742    -0.485071          0.299370   -0.163299        -0.114708   
1146    -0.485071          0.299370   -0.163299        -0.114708   
1759    -0.485071          0.299370   -0.163299        -0.114708   
...           ...               ...         ...              ...   
1249    -0.485071          0.299370   -0.163299         8.717798   
1604    -0.485

In [232]:
all_x = pd.concat((X_train_i, X_test_i),axis=0)
all_y = pd.concat((y_train_i, y_test_i),axis=0)
ttl_preds_i = list(zip(improved.PID, all_y.index, svm.predict(all_x),))
ttl_preds_i

[(534451170, 1509, 93.29186293008685),
 (903231060, 238, 67.05044199775125),
 (903204095, 1111, 90.15629406052247),
 (903228070, 678, 92.04904622840128),
 (903225160, 486, 78.95974220195941),
 (903230100, 214, 69.26859260244439),
 (534451150, 324, 67.96998016206624),
 (903233220, 381, 78.88411239121852),
 (903228060, 676, 71.84137561034117),
 (903231190, 1281, 108.37451782485829),
 (903231030, 469, 71.83075895984297),
 (903204040, 660, 87.3884859928287),
 (903202170, 857, 88.30837630783653),
 (902205010, 1820, 106.01123127379712),
 (903430130, 1602, 100.94673602637863),
 (902331010, 1464, 102.96515716069179),
 (902207130, 1187, 100.90829774566383),
 (902207110, 1994, 114.85294645326405),
 (535379060, 1927, 112.1957845230322),
 (902402260, 511, 86.05222411238113),
 (902205020, 114, 73.66175872264968),
 (902427040, 1360, 113.5806490142676),
 (902401090, 1662, 90.39958110177433),
 (535383070, 179, 70.4694543996229),
 (902134060, 1449, 99.33159892968155),
 (902202150, 1063, 81.862206847226

In [248]:
pred_df1 = pd.DataFrame((np.array(ttl_preds_i)),columns=['PID', 'd', 'Improved Prediction'])

In [249]:
pred_df1 = pred_df1.drop('d', axis=1)

In [254]:
pred_df1['PID']= pred_df1['PID'].astype(np.int64)

In [242]:
undervalued

Unnamed: 0        PID  GrLivArea  SalePrice MSSubClass MSZoning  \
6            550  534451170        729      52000         30       RL   
10           457  903231060        800      60000         30       RM   
24           182  903204095        935      79500         40       RM   
38           407  903228070       1142      93000         50       RM   
54           375  903225160       1290     106900         70       RM   
...          ...        ...        ...        ...        ...      ...   
2011          37  528188090       1626     155000        160       RL   
2012         256  528188060       1626     173000        160       RL   
2013         262  528188120       1626     175500        160       RL   
2014         807  528188150       1626     154000        160       RL   
2041          60  533135020       1755     235000         60       RL   

      LotFrontage  LotArea Street       Alley  LotShape LandContour LotConfig  \
6            56.0     4130   Pave  NoneListed         2         Lvl    Inside   
10           51.0     6120   Pave  NoneListed         3         Lvl    Inside   
24           50.0     9100   Pave  NoneListed         3         Lvl    Inside   
38           60.0     6000   Pave  NoneListed         3         Lvl    Corner   
54           59.0     5870   Pave  NoneListed         3         Lvl    Corner   
...           ...      ...    ...         ...       ...         ...       ...   
2011         50.0     2448   Pave  NoneListed         3         Lvl    Inside   
2012         98.0     2268   Pave  NoneListed         3         Lvl    Inside   
2013         85.0     2628   Pave  NoneListed         3         Lvl    Inside   
2014         36.0     2448   Pave  NoneListed         3         Lvl    Inside   
2041         70.0    11949   Pave  NoneListed         3         Lvl    Inside   

      LandSlope Neighborhood Condition1 Condition2 BldgType HouseStyle  \
6             2      BrkSide       Norm       Norm     1Fam     1Story   
10            2      BrkSide       Norm       Norm     1Fam     1Story   
24            2      BrkSide       RRAn      Feedr     1Fam     1Story   
38            2      BrkSide       Norm       Norm     1Fam     1.5Fin   
54            2      BrkSide      Feedr      Feedr     1Fam     2Story   
...         ...          ...        ...        ...      ...        ...   
2011          2      NridgHt       Norm       Norm    Twnhs     2Story   
2012          2      NridgHt       Norm       Norm    Twnhs     2Story   
2013          2      NridgHt       Norm       Norm    Twnhs     2Story   
2014          2      NridgHt       Norm       Norm    Twnhs     2Story   
2041          2      NoRidge       Norm       Norm     1Fam     2Story   

      OverallQual  OverallCond  YearBuilt  YearRemodAdd RoofStyle RoofMatl  \
6               3            6       1935          2003     Gable  CompShg   
10              2            3       1936          1950     Gable  CompShg   
24              5            6       1930          1960     Gable  CompShg   
38              5            5       1924          1950     Gable  CompShg   
54              6            9       1900          2000     Gable  CompShg   
...           ...          ...        ...           ...       ...      ...   
2011            7            5       2003          2004     Gable  CompShg   
2012            7            5       2003          2004     Gable  CompShg   
2013            7            5       2003          2003     Gable  CompShg   
2014            7            5       2003          2004     Gable  CompShg   
2041            7            8       1991          2008     Gable  CompShg   

     Exterior1st Exterior2nd MasVnrType  MasVnrArea  ExterQual  ExterCond  \
6        VinylSd     VinylSd       None         0.0          4          3   
10       AsbShng     AsbShng       None         0.0          2          2   
24       VinylSd     VinylSd       None         0.0          3          3   
38        Stucco      

In [256]:
total= undervalued.merge(pred_df1, on='PID')

In [267]:
total

Unnamed: 0        PID  GrLivArea  SalePrice MSSubClass MSZoning  \
0           550  534451170        729      52000         30       RL   
1           457  903231060        800      60000         30       RM   
2           182  903204095        935      79500         40       RM   
3           407  903228070       1142      93000         50       RM   
4           375  903225160       1290     106900         70       RM   
..          ...        ...        ...        ...        ...      ...   
303          37  528188090       1626     155000        160       RL   
304         256  528188060       1626     173000        160       RL   
305         262  528188120       1626     175500        160       RL   
306         807  528188150       1626     154000        160       RL   
307          60  533135020       1755     235000         60       RL   

     LotFrontage  LotArea Street       Alley  LotShape LandContour LotConfig  \
0           56.0     4130   Pave  NoneListed         2         Lvl    Inside   
1           51.0     6120   Pave  NoneListed         3         Lvl    Inside   
2           50.0     9100   Pave  NoneListed         3         Lvl    Inside   
3           60.0     6000   Pave  NoneListed         3         Lvl    Corner   
4           59.0     5870   Pave  NoneListed         3         Lvl    Corner   
..           ...      ...    ...         ...       ...         ...       ...   
303         50.0     2448   Pave  NoneListed         3         Lvl    Inside   
304         98.0     2268   Pave  NoneListed         3         Lvl    Inside   
305         85.0     2628   Pave  NoneListed         3         Lvl    Inside   
306         36.0     2448   Pave  NoneListed         3         Lvl    Inside   
307         70.0    11949   Pave  NoneListed         3         Lvl    Inside   

     LandSlope Neighborhood Condition1 Condition2 BldgType HouseStyle  \
0            2      BrkSide       Norm       Norm     1Fam     1Story   
1            2      BrkSide       Norm       Norm     1Fam     1Story   
2            2      BrkSide       RRAn      Feedr     1Fam     1Story   
3            2      BrkSide       Norm       Norm     1Fam     1.5Fin   
4            2      BrkSide      Feedr      Feedr     1Fam     2Story   
..         ...          ...        ...        ...      ...        ...   
303          2      NridgHt       Norm       Norm    Twnhs     2Story   
304          2      NridgHt       Norm       Norm    Twnhs     2Story   
305          2      NridgHt       Norm       Norm    Twnhs     2Story   
306          2      NridgHt       Norm       Norm    Twnhs     2Story   
307          2      NoRidge       Norm       Norm     1Fam     2Story   

     OverallQual  OverallCond  YearBuilt  YearRemodAdd RoofStyle RoofMatl  \
0              3            6       1935          2003     Gable  CompShg   
1              2            3       1936          1950     Gable  CompShg   
2              5            6       1930          1960     Gable  CompShg   
3              5            5       1924          1950     Gable  CompShg   
4              6            9       1900          2000     Gable  CompShg   
..           ...          ...        ...           ...       ...      ...   
303            7            5       2003          2004     Gable  CompShg   
304            7            5       2003          2004     Gable  CompShg   
305            7            5       2003          2003     Gable  CompShg   
306            7            5       2003          2004     Gable  CompShg   
307            7            8       1991          2008     Gable  CompShg   

    Exterior1st Exterior2nd MasVnrType  MasVnrArea  ExterQual  ExterCond  \
0       VinylSd     VinylSd       None         0.0          4          3   
1       AsbShng     AsbShng       None         0.0          2          2   
2       VinylSd     VinylSd       None         0.0          3          3   
3        Stucco      Stucco    BrkFace       444.0          3          3 

In [259]:
total['Improved Saleprice'] = total['Improved Prediction'] * total['GrLivArea']

In [260]:
total['Gained_SP'] = total['Improved Saleprice'] - total['SalePricePerGLA'] * total['GrLivArea']

In [271]:
total1 = total[['PID', 'SalePrice', 'Predicted SalePrice' , 'Improved Saleprice', 'Gained_SP']]

In [287]:
top= total1.sort_values(ascending= False, by= 'Gained_SP').head(50)

#### Top 50 improved SalePrices

In [288]:
top

PID  SalePrice  Predicted SalePrice  Improved Saleprice  \
49   911102170      40000        128783.841251       144679.576967   
72   908102130     100000        132423.820414       175381.853215   
53   903484080      50000        181850.962839       124014.841518   
16   902207130      12789         68844.361795        83955.703724   
33   902103090     111500        178562.387598       180595.400269   
41   909250210     102000        198674.335859       171037.401392   
105  905200290     155000        235537.130490       222860.427953   
232  914476020     126000        157385.177484       193396.864352   
161  534427010      84900        205178.436373       151260.138728   
170  535456050     130000        237605.517201       196101.135201   
234  923202060     115000        227411.104251       177782.084005   
67   905480210     108000        173930.365369       170101.274118   
34   902109130      94550        260376.618009       156589.882357   
163  535150070     104900        271976.584334       165189.649019   
74   905450020     127500        214983.756517       185661.293223   
65   905480160      91500        276977.271828       149279.671197   
70   909101140     117000        241272.956378       173561.754354   
225  923125030      81500        210510.504256       137578.133757   
26   902325160      79000        212504.711525       135056.111835   
171  535457040     141000        157900.164844       196982.092857   
97   904100170     131000        219442.100352       186154.688848   
21   902427040      87000        149071.417624       141748.649970   
78   905478030     131000        243819.435229       185561.177851   
63   904351200     103600        184204.520150       156442.679549   
37   902305090      87000        178724.089776       135703.227186   
111  909250120     138887        283635.541727       187438.947545   
60   908204180      61500        219257.546484       109774.054849   
103  905101310     136500        191042.211176       184602.316489   
43   909250030     100000        240298.374995       146768.800643   
61   908204160      82500         81745.510670       128869.662071   
160  534225120     142600        222119.075982       188320.349047   
235  923204150     139000        240206.897878       183752.432324   
18   535379060      64500         81811.192348       108605.519418   
128  535180130     108000        155899.228923       150508.585530   
48   910205020      78000        183644.100243       120232.837919   
58   905480150      79000        161748.033175       121004.861328   
100  532376090     143900        283160.301126       184915.418485   
52   903458110      89500        142209.335814       129824.788136   
131  534479120     105000        197227.983500       144780.885827   
24   902134060      96900        243401.545157       136680.280127   
257  905226110     145000        189115.845814       184024.263015   
159  535354260     156000        252775.003121       194803.444844   
132  535126100     110000        175008.021993       148552.597888   
230  914452060     145000        264833.155445       183516.775286   
19   902402260      55000         73879.452917        93452.715386   
62   905480090      96000        108614.942812       133866.077247   
252  527451020     106000        234099.050010       143460.483907   
29   902109080     118000        199859.547678       154734.270031   
51   911102090      89000         91402.648783       125681.535600   
256  905351045     148400        199294.578360       184629.225646   

         Gained_SP  
49   104679.576967  
72    75381.853215  
53    74014.841518  
16    71166.703724  
33    69095.400269  
41    69037.401392  
105   67860.427953  
232   67396.864352  
161   66360.138728  
170   66101.135201  
234   62782.084005  
67    62101.274118  
34    62039.882357  
163   60289.649019  
74    58161.293223  
65    57779.671197  
70    56561.754354  
225   56078.133757  
26    56056.111835  
171   55982.092857  